# Importing Libraries
This cell imports the libraries needed for video processing, hand detection, and image manipulation.

In [ ]:
import cv2
import mediapipe as mp
import time
import numpy as np
from IPython.display import display
from PIL import Image
from colorama import Fore

# MediaPipe Hand Detection Configuration
This section sets up hand detection using the MediaPipe library with parameters adjusted for higher accuracy.

In [ ]:
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(static_image_mode=False, max_num_hands=2, min_detection_confidence=0.7, min_tracking_confidence=0.7)

# Control variables
blue_grip = False
white_grip = False
blue_start_time = 0
white_start_time = 0
blue_disappear_time = 0
white_disappear_time = 0
disappearance_tolerance = 2
winner = None

# Function to Check if the Grip Was Held for 7 Seconds
This function checks if the grip was held for 7 seconds, returning `True` or `False`.

In [ ]:
def check_grip(start_time, current_time):
    return (current_time - start_time) >= 7

# Video Initialization
This section configures the video path and starts the frame capture process.

In [ ]:
video_path = 'assets/videos/test_0.mp4'
cap = cv2.VideoCapture(video_path)

# Auxiliary Functions
These functions assist in detecting the colors of the kimonos and drawing the status on the frame.

In [ ]:
def detect_kimono_color(frame_hsv):
    blue_lower = np.array([100, 150, 50])
    blue_upper = np.array([140, 255, 255])
    white_lower = np.array([0, 0, 200])
    white_upper = np.array([180, 30, 255])
    mask_blue = cv2.inRange(frame_hsv, blue_lower, blue_upper)
    mask_white = cv2.inRange(frame_hsv, white_lower, white_upper)
    athlete_blue = cv2.countNonZero(mask_blue) > 1000
    athlete_white = cv2.countNonZero(mask_white) > 1000
    return athlete_blue, athlete_white

# Function to Draw Status on the Frame
This function draws the status of the grips on the frame, indicating whether the blue or white athlete's grip is active.

In [ ]:
def draw_status(frame, blue_grip, white_grip):
    cv2.putText(frame, f'Blue Grip: {'Active' if blue_grip else 'Inactive'}', 
                (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)
    cv2.putText(frame, f'White Grip: {'Active' if white_grip else 'Inactive'}', 
                (10, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)

# Video Processing
Main loop to process each video frame, detect the grip, and determine the winner.

In [ ]:
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        print('End of video or error reading frame.')
        break

    # Convert to RGB and HSV
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    frame_hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

    # Detect kimono color
    athlete_blue, athlete_white = detect_kimono_color(frame_hsv)

    # Process the frame for hand detection
    results = hands.process(frame_rgb)

    # Get current time
    current_time = time.time()

    # Check if hands are detected
    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            mp.solutions.drawing_utils.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

    # Check if the grip was held for 7 seconds
    if check_grip(blue_start_time, current_time) and winner is None:
        print(Fore.BLUE + 'Blue athlete won the grip!')
        winner = 'blue' # Mark that the blue athlete won

    if check_grip(white_start_time, current_time) and winner is None:
        print(Fore.RED + 'White athlete won the grip!')
        winner = 'white' # Mark that the white athlete won

    # Update status and display the frame in Jupyter
    draw_status(frame, athlete_blue, athlete_white)

    # Show the frame
    display(Image.fromarray(frame)) # Display directly as BGR

    # Adding a small delay for Jupyter to process the display
    time.sleep(0.05)

cap.release()
